In [1]:
import numpy as np
import networkx as nx
import os
import pandas as pd
from networkx.algorithms.community import girvan_newman
import community as community_louvain
from sklearn.preprocessing import StandardScaler
from itertools import islice
import itertools
from sklearn.cluster import KMeans
from sklearn.manifold import Isomap
from networkx.algorithms.community import modularity

# Newman-Girvan

In [2]:
def load_anatomical_mappings(file_path):
    # Load the anatomical locations from the CSV file
    anatomical_loc_df = pd.read_csv(file_path)
    
    # Ensure 'Channel' column is integer type, then adjust for zero-based index
    anatomical_loc_df['Channel'] = anatomical_loc_df['Channel'].astype(int) - 1
    
    # Create dictionaries for mapping channel to anatomical information
    channel_to_label = dict(zip(anatomical_loc_df['Channel'], anatomical_loc_df['Label_name']))
    channel_to_lobe = dict(zip(anatomical_loc_df['Channel'], anatomical_loc_df['Lobe']))
    channel_to_hem = dict(zip(anatomical_loc_df['Channel'], anatomical_loc_df['Hemisphere']))
    
    return channel_to_label, channel_to_lobe, channel_to_hem

def detect_communities_girvan_newman(graph):
    # Detect communities using the Girvan-Newman algorithm
    comp = girvan_newman(graph)
    
    # Iterate to find the split with exactly 5 communities
    for communities in comp:
        if len(communities) >= 5:
            communities = list(itertools.islice(communities, 5))
            break
    
    return communities

def compute_cooccurrence_matrix(community_list, num_nodes):
    cooccurrence_matrix = np.zeros((num_nodes, num_nodes))
    
    for communities in community_list:
        for community in communities:
            community = list(community)
            for i, node1 in enumerate(community):
                node1 = int(node1)  # Ensure node1 is an integer
                if node1 < 0 or node1 >= num_nodes:
                    print(f"Invalid node index: {node1}")
                    continue
                for node2 in community[i + 1:]:
                    node2 = int(node2)  # Ensure node2 is an integer
                    if node2 < 0 or node2 >= num_nodes:
                        print(f"Invalid node index: {node2}")
                        continue
                    cooccurrence_matrix[node1, node2] += 1
                    cooccurrence_matrix[node2, node1] += 1
    
    return cooccurrence_matrix

def perform_kmeans_clustering(cooccurrence_matrix, num_clusters):
    # Normalize the co-occurrence matrix 
    scaler = StandardScaler()
    normalized_matrix = scaler.fit_transform(cooccurrence_matrix)
    
    # Perform K-Means clustering
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(normalized_matrix)
    
    return cluster_labels

def save_final_communities(contrast_id, cluster_labels, channel_to_label, channel_to_lobe, channel_to_hem):
    # Prepare to save the communities
    community_data = []
    for node, community_id in enumerate(cluster_labels):
        label_name = channel_to_label.get(int(node), 'Unknown')
        lobe = channel_to_lobe.get(int(node), 'Unknown')
        hemisphere = channel_to_hem.get(int(node), 'Unknown')
        community_data.append({
            'contrast_id': contrast_id,
            'community_id': community_id,
            'node': node,
            'label_name': label_name,
            'lobe': lobe,
            'hemisphere': hemisphere
        })
    
    # Convert to DataFrame
    community_df = pd.DataFrame(community_data)
    
    # Sort by 'community_id'
    community_df.sort_values(by='community_id', inplace=True)
    
    # Save to CSV
    community_df.to_csv(f'communities_girvan_newman/contrast_{contrast_id}_final_communities.csv', index=False)
    
    print(f"Final communities for contrast {contrast_id} saved to 'communities_girvan_newman/contrast_{contrast_id}_final_communities.csv'.")

def main():
    anatomical_file_path = 'fNIRS channels anatomical labels.csv' 
    channel_to_label, channel_to_lobe, channel_to_hem = load_anatomical_mappings(anatomical_file_path)
    
    num_subjects = 98  
    num_contrasts = 3  
    num_nodes = 64 
    num_clusters = 5  # Set the number of clusters (communities) for K-Means

    os.makedirs('communities_girvan_newman', exist_ok=True)

    for contrast in range(num_contrasts):
        community_list = []
        
        for subject_id in range(1, num_subjects + 1):
            graph_path = f'graphs/{subject_id}_contrast_{contrast + 1}_network.graphml'
            if not os.path.exists(graph_path):
                print(f"Graph file {graph_path} does not exist, skipping.")
                continue
            
            graph = nx.read_graphml(graph_path)
            communities = detect_communities_girvan_newman(graph)
            if communities:
                community_list.append(communities)
        
        # Compute the co-occurrence matrix for this contrast
        cooccurrence_matrix = compute_cooccurrence_matrix(community_list, num_nodes)
        
        # Convert the co-occurrence matrix to integers
        cooccurrence_matrix = cooccurrence_matrix.astype(int)
        
        # Save the co-occurrence matrix to CSV
        nodes = list(range(num_nodes))
        cooccurrence_df = pd.DataFrame(cooccurrence_matrix, index=nodes, columns=nodes)
        cooccurrence_df.to_csv(f"cooccurrence_matrix_contrast_{contrast + 1}_girvan_newman.csv", float_format='%.0f')  # Save as integers
        
        print(f"Co-occurrence matrix saved to 'cooccurrence_matrix_contrast_{contrast + 1}_girvan_newman.csv'.")
        
        # Use K-Means clustering on the co-occurrence matrix to detect final communities
        kmeans_labels = perform_kmeans_clustering(cooccurrence_matrix, num_clusters)
        save_final_communities(contrast + 1, kmeans_labels, channel_to_label, channel_to_lobe, channel_to_hem)

if __name__ == "__main__":
    main()

Graph file graphs/1_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/2_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/3_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/4_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/5_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/6_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/7_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/8_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/9_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/10_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/11_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/23_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/27_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/28_contrast_1_network.graphml

/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Final communities for contrast 1 saved to 'communities_girvan_newman/contrast_1_final_communities.csv'.
Graph file graphs/1_contrast_2_network.graphml does not exist, skipping.
Graph file graphs/2_contrast_2_network.graphml does not exist, skipping.
Graph file graphs/3_contrast_2_network.graphml does not exist, skipping.
Graph file graphs/4_contrast_2_network.graphml does not exist, skipping.
Graph file graphs/5_contrast_2_network.graphml does not exist, skipping.
Graph file graphs/6_contrast_2_network.graphml does not exist, skipping.
Graph file graphs/7_contrast_2_network.graphml does not exist, skipping.
Graph file graphs/8_contrast_2_network.graphml does not exist, skipping.
Graph file graphs/9_contrast_2_network.graphml does not exist, skipping.
Graph file graphs/10_contrast_2_network.graphml does not exist, skipping.
Graph file graphs/11_contrast_2_network.graphml does not exist, skipping.
Graph file graphs/23_contrast_2_network.graphml does not exist, skipping.
Graph file graphs

/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Graph file graphs/23_contrast_3_network.graphml does not exist, skipping.
Graph file graphs/27_contrast_3_network.graphml does not exist, skipping.
Graph file graphs/28_contrast_3_network.graphml does not exist, skipping.
Graph file graphs/29_contrast_3_network.graphml does not exist, skipping.
Graph file graphs/30_contrast_3_network.graphml does not exist, skipping.
Graph file graphs/31_contrast_3_network.graphml does not exist, skipping.
Graph file graphs/36_contrast_3_network.graphml does not exist, skipping.
Co-occurrence matrix saved to 'cooccurrence_matrix_contrast_3_girvan_newman.csv'.
Final communities for contrast 3 saved to 'communities_girvan_newman/contrast_3_final_communities.csv'.


/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


# Louvain method

In [3]:
def load_anatomical_mappings(file_path):
    # Load the anatomical locations from the CSV file
    anatomical_loc_df = pd.read_csv(file_path)
    
    # Create dictionaries for mapping channel to anatomical information
    channel_to_label = dict(zip(anatomical_loc_df['Channel'] - 1, anatomical_loc_df['Label_name']))  # Adjust for zero-based index
    channel_to_lobe = dict(zip(anatomical_loc_df['Channel'] - 1, anatomical_loc_df['Lobe']))  # Adjust for zero-based index
    channel_to_hem = dict(zip(anatomical_loc_df['Channel'] - 1, anatomical_loc_df['Hemisphere']))  # Adjust for zero-based index
    
    return channel_to_label, channel_to_lobe, channel_to_hem

def detect_communities_louvain(graph):
    # Detect communities using the Louvain method
    partition = community_louvain.best_partition(graph)
    communities = {}
    for node, community_id in partition.items():
        if community_id not in communities:
            communities[community_id] = []
        communities[community_id].append(node)
    return list(communities.values())

def compute_cooccurrence_matrix(community_list, num_nodes):
    cooccurrence_matrix = np.zeros((num_nodes, num_nodes))
    
    for communities in community_list:
        for community in communities:
            community = list(community)
            for i, node1 in enumerate(community):
                node1 = int(node1)  # Ensure node1 is an integer
                if node1 < 0 or node1 >= num_nodes:
                    print(f"Invalid node index: {node1}")
                    continue
                for node2 in community[i + 1:]:
                    node2 = int(node2)  # Ensure node2 is an integer
                    if node2 < 0 or node2 >= num_nodes:
                        print(f"Invalid node index: {node2}")
                        continue
                    cooccurrence_matrix[node1, node2] += 1
                    cooccurrence_matrix[node2, node1] += 1
    
    return cooccurrence_matrix

def save_final_communities(contrast_id, cooccurrence_matrix, channel_to_label, channel_to_lobe, channel_to_hem, num_final_communities=5):
    # Apply K-Means to the co-occurrence matrix
    kmeans = KMeans(n_clusters=num_final_communities, random_state=0)
    final_communities = kmeans.fit_predict(cooccurrence_matrix)
    
    # Prepare to save the communities
    community_data = []
    for node, community_id in enumerate(final_communities):
        label_name = channel_to_label.get(int(node), 'Unknown')
        lobe = channel_to_lobe.get(int(node), 'Unknown')
        hemisphere = channel_to_hem.get(int(node), 'Unknown')
        community_data.append({
            'contrast_id': contrast_id,
            'community_id': community_id,
            'node': node,
            'label_name': label_name,
            'lobe': lobe,
            'hemisphere': hemisphere
        })
    
    # Convert to DataFrame and sort by community_id
    community_df = pd.DataFrame(community_data)
    community_df = community_df.sort_values(by='community_id')
    
    # Save the sorted DataFrame to a CSV file
    community_df.to_csv(f'communities_louvain/contrast_{contrast_id}_final_communities.csv', index=False)
    print(f"Final communities for contrast {contrast_id} saved to 'communities_louvain/contrast_{contrast_id}_final_communities.csv'.")

def main():
    anatomical_file_path = 'fNIRS channels anatomical labels.csv'
    channel_to_label, channel_to_lobe, channel_to_hem = load_anatomical_mappings(anatomical_file_path)
    
    num_subjects = 98  
    num_contrasts = 3 
    num_nodes = 64  
    
    os.makedirs('communities_louvain', exist_ok=True)

    for contrast in range(num_contrasts):
        community_list = []
        
        for subject_id in range(1, num_subjects + 1):
            graph_path = f'graphs/{subject_id}_contrast_{contrast + 1}_network.graphml'
            if not os.path.exists(graph_path):
                print(f"Graph file {graph_path} does not exist, skipping.")
                continue
            
            graph = nx.read_graphml(graph_path)
            communities = detect_communities_louvain(graph)
            if communities:
                community_list.append(communities)
        
        # Compute the co-occurrence matrix for this contrast
        cooccurrence_matrix = compute_cooccurrence_matrix(community_list, num_nodes)
        
        # Save the co-occurrence matrix to CSV
        nodes = list(range(num_nodes))
        cooccurrence_df = pd.DataFrame(cooccurrence_matrix, index=nodes, columns=nodes)
        cooccurrence_df.to_csv(f"cooccurrence_matrix_contrast_{contrast + 1}_louvain.csv", float_format='%.0f')  # Save as integers
        print(f"Co-occurrence matrix saved to 'cooccurrence_matrix_contrast_{contrast + 1}_louvain.csv'.")
        
        # Use the co-occurrence matrix to save the final communities
        save_final_communities(contrast + 1, cooccurrence_matrix, channel_to_label, channel_to_lobe, channel_to_hem)

if __name__ == "__main__":
    main()

Graph file graphs/1_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/2_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/3_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/4_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/5_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/6_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/7_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/8_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/9_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/10_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/11_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/23_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/27_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/28_contrast_1_network.graphml

/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Co-occurrence matrix saved to 'cooccurrence_matrix_contrast_2_louvain.csv'.
Final communities for contrast 2 saved to 'communities_louvain/contrast_2_final_communities.csv'.
Graph file graphs/1_contrast_3_network.graphml does not exist, skipping.
Graph file graphs/2_contrast_3_network.graphml does not exist, skipping.
Graph file graphs/3_contrast_3_network.graphml does not exist, skipping.
Graph file graphs/4_contrast_3_network.graphml does not exist, skipping.
Graph file graphs/5_contrast_3_network.graphml does not exist, skipping.
Graph file graphs/6_contrast_3_network.graphml does not exist, skipping.
Graph file graphs/7_contrast_3_network.graphml does not exist, skipping.
Graph file graphs/8_contrast_3_network.graphml does not exist, skipping.
Graph file graphs/9_contrast_3_network.graphml does not exist, skipping.
Graph file graphs/10_contrast_3_network.graphml does not exist, skipping.
Graph file graphs/11_contrast_3_network.graphml does not exist, skipping.
Graph file graphs/23_

/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Co-occurrence matrix saved to 'cooccurrence_matrix_contrast_3_louvain.csv'.
Final communities for contrast 3 saved to 'communities_louvain/contrast_3_final_communities.csv'.


/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


# Isomap

In [4]:
def load_anatomical_mappings(file_path):
    # Load the anatomical locations from the CSV file
    anatomical_loc_df = pd.read_csv(file_path)
    
    # Create dictionaries for mapping channel to anatomical information
    channel_to_label = dict(zip(anatomical_loc_df['Channel'] - 1, anatomical_loc_df['Label_name']))  # Adjust for zero-based index
    channel_to_lobe = dict(zip(anatomical_loc_df['Channel'] - 1, anatomical_loc_df['Lobe']))  # Adjust for zero-based index
    channel_to_hem = dict(zip(anatomical_loc_df['Channel'] - 1, anatomical_loc_df['Hemisphere']))  # Adjust for zero-based index
    
    return channel_to_label, channel_to_lobe, channel_to_hem

def detect_communities_isomap(graph, n_components=3, n_clusters=5):  
    # Get the adjacency matrix of the graph
    adjacency_matrix = nx.to_numpy_array(graph)
    
    # Apply Isomap for dimensionality reduction
    isomap = Isomap(n_components=n_components)
    embedding = isomap.fit_transform(adjacency_matrix)
    
    # Cluster the nodes in the reduced-dimensional space
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    community_ids = kmeans.fit_predict(embedding)
    
    # Group nodes by community
    communities = {}
    for node, community_id in enumerate(community_ids):
        if community_id not in communities:
            communities[community_id] = []
        communities[community_id].append(node)
    
    return list(communities.values())

def compute_cooccurrence_matrix(community_list, num_nodes):
    cooccurrence_matrix = np.zeros((num_nodes, num_nodes))
    
    for communities in community_list:
        for community in communities:
            for i, node1 in enumerate(community):
                for node2 in community[i + 1:]:
                    cooccurrence_matrix[node1, node2] += 1
                    cooccurrence_matrix[node2, node1] += 1
    
    return cooccurrence_matrix

def save_final_communities(contrast_id, cooccurrence_matrix, channel_to_label, channel_to_lobe, channel_to_hem, num_final_communities=5):
    # Apply K-Means to the co-occurrence matrix
    kmeans = KMeans(n_clusters=num_final_communities, random_state=0)
    final_communities = kmeans.fit_predict(cooccurrence_matrix)
    
    # Prepare to save the communities
    community_data = []
    for node, community_id in enumerate(final_communities):
        label_name = channel_to_label.get(int(node), 'Unknown')
        lobe = channel_to_lobe.get(int(node), 'Unknown')
        hemisphere = channel_to_hem.get(int(node), 'Unknown')
        community_data.append({
            'contrast_id': contrast_id,
            'community_id': community_id,
            'node': node,
            'label_name': label_name,
            'lobe': lobe,
            'hemisphere': hemisphere
        })
    
    # Convert to DataFrame and sort by community_id
    community_df = pd.DataFrame(community_data)
    community_df = community_df.sort_values(by='community_id')
    
    # Save the sorted DataFrame to a CSV file
    community_df.to_csv(f'communities_isomap/contrast_{contrast_id}_final_communities.csv', index=False)
    print(f"Final communities for contrast {contrast_id} saved to 'communities_isomap/contrast_{contrast_id}_final_communities.csv'.")

def main():
    anatomical_file_path = 'fNIRS channels anatomical labels.csv'
    channel_to_label, channel_to_lobe, channel_to_hem = load_anatomical_mappings(anatomical_file_path)
    
    num_subjects = 98  
    num_contrasts = 3  
    num_nodes = 64 
    
    os.makedirs('communities_isomap', exist_ok=True)

    for contrast in range(num_contrasts):
        community_list = []
        
        for subject_id in range(1, num_subjects + 1):
            graph_path = f'graphs/{subject_id}_contrast_{contrast + 1}_network.graphml'
            if not os.path.exists(graph_path):
                print(f"Graph file {graph_path} does not exist, skipping.")
                continue
            
            graph = nx.read_graphml(graph_path)
            communities = detect_communities_isomap(graph, n_components=3)  
            if communities:
                community_list.append(communities)
        
        # Compute the co-occurrence matrix for this contrast
        cooccurrence_matrix = compute_cooccurrence_matrix(community_list, num_nodes)
        
        # Save the co-occurrence matrix to CSV
        nodes = list(range(num_nodes))
        cooccurrence_df = pd.DataFrame(cooccurrence_matrix, index=nodes, columns=nodes)
        cooccurrence_df.to_csv(f"cooccurrence_matrix_contrast_{contrast + 1}_isomap.csv", float_format='%.0f')  # Save as integers
        print(f"Co-occurrence matrix saved to 'cooccurrence_matrix_contrast_{contrast + 1}_isomap.csv'.")
        
        # Use the co-occurrence matrix to save the final communities
        save_final_communities(contrast + 1, cooccurrence_matrix, channel_to_label, channel_to_lobe, channel_to_hem)

if __name__ == "__main__":
    main()

Graph file graphs/1_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/2_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/3_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/4_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/5_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/6_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/7_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/8_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/9_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/10_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/11_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/23_contrast_1_network.graphml does not exist, skipping.


/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default valu

Graph file graphs/27_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/28_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/29_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/30_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/31_contrast_1_network.graphml does not exist, skipping.
Graph file graphs/36_contrast_1_network.graphml does not exist, skipping.


/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default valu

/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default valu

...

Graph file graphs/27_contrast_3_network.graphml does not exist, skipping.
Graph file graphs/28_contrast_3_network.graphml does not exist, skipping.
Graph file graphs/29_contrast_3_network.graphml does not exist, skipping.
Graph file graphs/30_contrast_3_network.graphml does not exist, skipping.
Graph file graphs/31_contrast_3_network.graphml does not exist, skipping.
Graph file graphs/36_contrast_3_network.graphml does not exist, skipping.


/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default valu

/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default valu

Co-occurrence matrix saved to 'cooccurrence_matrix_contrast_3_isomap.csv'.
Final communities for contrast 3 saved to 'communities_isomap/contrast_3_final_communities.csv'.


/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/paruljain/opt/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
